In [1]:
from Error_Compare import *
import analysis.dtw as dtw
import numpy as np
%load_ext autoreload
%autoreload 2

Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [2]:
def ptp_abs_error(live, pp):
    # Live will be the warped data aligned to the x_times of pp
    error = np.zeros([len(live),2])
    for i in range(len(live)):
        x = live[i,0]
        # get pp_val at x
        pp_val = pp[pp[:,0] == x,1]
        error[i,:] = x,abs(live[i,1] - pp_val)
    return error

def ptp_percent_error(live, pp):
    # Live will be the warped data aligned to the x_times of pp
    error = np.zeros([len(live),2])
    for i in range(len(live)):
        x = live[i,0]
        # get pp_val at x
        pp_val = pp[pp[:,0] == x,1]
        error[i,:] = x,abs(live[i,1] - pp_val)/min(pp_val, live[i,1])
        # fill any nan values with last value
        error[np.isnan(error)] = error[np.isnan(error).sum(axis = 1) > 0,-1]
    return error

def get_frac_in_error(error_signal, low, high = np.inf):
    error_indices = error_signal[(error_signal[:,1] > low) & (error_signal[:,1] < high),0]
    #error_values = error_signal[(error_signal[:,1] > low) & (error_signal[:,1] < high),1]
    frac_in_error = len(error_indices)/len(error_signal)
    return frac_in_error, error_indices #error_values

def ptp_missed_peak_error(live, pp):
    error = np.zeros([len(live),2])
    for i in range(len(live)):
        x = live[i,0]
        # get pp_val at x
        pp_val = pp[pp[:,0] == x,1]
        error[i,:] = x, abs(live[i,1] - pp_val)/live[i,1]
        # fill any nan values with last value
        error[np.isnan(error)] = error[np.isnan(error).sum(axis = 1) > 0,-1]
    return error



In [3]:
activity_dir = "data/dataset/2b8f256d-7063-410c-b66d-3bdfb7d140c5"
# Timeframe to look at
min_time = 120
live_min_time = 130 # we want a slightly delayed start for the live data
max_time = np.inf
# DTW Settings
window1 = [1,10]
window2 = 3
gamma = 2 # scaling of shifting deviation penalty
beta = 0.2 # exponential averaging factor



In [4]:

activity = os.path.split(activity_dir)[-1]
clean_dfs = load_data(activity_dir, has_uncleaned = False)

key_metrics = ["breathTime", "VT", "VE", "instBR", "RRAvg"]#, "VT", "RRAvg", "instBR"]


# Load Data
raw_chest = clean_dfs["raw_slow_df"][["time","c",]]

live_b3_df = clean_dfs["live_b3_df"][key_metrics]
pp_b3_df = clean_dfs["aws_b3_df"][key_metrics]

#c_df = pd.read_csv("data/dataset/2b8f256d-7063-410c-b66d-3bdfb7d140c5/C_data/C_Offline_2023-05-12_vt1_ride_Raw.csv")



# Only look at a subset of the data
raw_chest = raw_chest[(raw_chest["time"] > min_time) & (raw_chest["time"] < max_time)]
live_b3_df = live_b3_df[(live_b3_df["breathTime"] > live_min_time) & (live_b3_df["breathTime"] < max_time)]
pp_b3_df = pp_b3_df[(pp_b3_df["breathTime"] > min_time) & (pp_b3_df["breathTime"] < max_time)]
#c_df = c_df[(c_df["ValleyTS"] > min_time) & (c_df["ValleyTS"] < max_time)]
# print(c_df)

No pickled data found in activity data directory. Cleaning data...
cleaned_data folder already exists at data/dataset\cleaned_data


In [5]:
# Check of pp_b3_df contains nans
print("pp_b3_df contains nans: ", pp_b3_df.isnull().values.any())
# Backfill nans
pp_b3_df = pp_b3_df.fillna(method = "bfill")

pp_b3_df contains nans:  True


In [6]:
# First convert to numpy arrays
VT_live = live_b3_df[["breathTime", "VT"]].to_numpy()
VT_pp = pp_b3_df[["breathTime", "VT"]].to_numpy()

In [7]:
VT_DTW, VT_i_to_j, _ = dtw.dtw_distance(VT_live, VT_pp, avg_shift= True, window = window1, gamma = gamma, beta = beta)
warped = dtw.remap_query(VT_live, VT_pp, VT_i_to_j)
VT_i_to_j_fixed = dtw.fix_i_to_j(VT_i_to_j, warped)
VT_DTW2, VT_i_to_j_2 = dtw.dtw2(VT_i_to_j_fixed, VT_pp, window2)

#rewarped = dtw.remap_query(VT_live, VT_pp, VT_i_to_j_fixed)

IndexError: invalid index to scalar variable.

In [310]:
VT_error1 = ptp_abs_error(warped, VT_pp)
VT_error2 = ptp_abs_error(rewarped, VT_pp)

VT_perror1 = ptp_percent_error(warped, VT_pp)
VT_perror2 = ptp_percent_error(rewarped, VT_pp)

VT_error1_frac, VT_error1_ind = get_frac_in_error(VT_error1, 100)
VT_error2_frac, VT_error2_ind = get_frac_in_error(VT_error2, 100)

VT_perror1_frac, VT_perror1_ind = get_frac_in_error(VT_perror1, .50)
VT_perror2_frac, VT_perror2_ind = get_frac_in_error(VT_perror2, .50)


VT_perror_avg = np.mean(VT_perror1[:,1])

VT_missed_peak_error = ptp_missed_peak_error(rewarped, VT_pp)
VT_missed_peak_error_frac, missed_peak_ind = get_frac_in_error(VT_missed_peak_error, 1.0)

#print("VT_error1_frac: ", VT_error1_frac)
print("VT_error2_frac: ", VT_error2_frac)

#print("VT_perror1_frac:", VT_perror1_frac)
print("VT_perror2_frac:", VT_perror2_frac)

print("VT_perror_avg:", VT_perror_avg)

print("Missed Peak frac:", VT_missed_peak_error_frac)




VT_error2_frac:  0.04497354497354497
VT_perror2_frac: 0.07936507936507936
VT_perror_avg: 0.23548964289498361
Missed Peak frac: 0.006613756613756613


In [311]:
# plot the error
fig = make_subplots(rows = 3, cols = 1, shared_xaxes = True, vertical_spacing = 0.05, subplot_titles = ("VT", "Abs Error"))
# First plot the VT
fig.add_trace(go.Scatter(x = VT_live[:,0], y = VT_live[:,1], marker = dict(color = "blue"),  name = "Live"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = VT_pp[:,0], y = VT_pp[:,1], marker = dict(color = "red"), name = "PP"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = warped[:,0], y = warped[:,1], line = dict(dash = "dash"),marker = dict(color = "blue"), name = "Warped"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = rewarped[:,0], y = rewarped[:,1], line = dict(dash = "dot"),marker = dict(color = "blue"), name = "Rewarped"), row = 1, col = 1)
# fig.add_trace(go.Scatter(x = c_df["PeakTS"], y = c_df["VT"], marker = dict(color = "green"), name = "C Detections"), row = 1, col = 1)
# Now plot the error
fig.add_trace(go.Scatter(x = VT_error1[:,0], y = VT_error1[:,1], line = dict(dash = "dash"), marker = dict(color = "green"),  name = "Warped Error"), row = 2, col = 1)
fig.add_trace(go.Scatter(x = VT_error2[:,0], y = VT_error2[:,1], line = dict(dash = "dot"), marker = dict(color = "green"), name = "Rewarped Error"), row=2, col = 1)

fig.add_trace(go.Scatter(x = raw_chest["time"], y = raw_chest["c"], marker = dict(color = "blue"), name = "Chest"), row = 3, col = 1)
fig.update_layout(title = "Tidal Volume (VT) Absolute Error")

In [312]:
# plot the error
fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, vertical_spacing = 0.05, subplot_titles = ("VT", "Percent Error"))
# First plot the VT
fig.add_trace(go.Scatter(x = VT_live[:,0], y = VT_live[:,1], marker = dict(color = "blue"),  name = "Live"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = VT_pp[:,0], y = VT_pp[:,1], marker = dict(color = "red"), name = "PP"), row = 1, col = 1)
# fig.add_trace(go.Scatter(x = warped[:,0], y = warped[:,1], line = dict(dash = "dash"),marker = dict(color = "blue"), name = "Warped"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = rewarped[:,0], y = rewarped[:,1], line = dict(dash = "dot"),marker = dict(color = "blue"), name = "Rewarped"), row = 1, col = 1)

# Now plot the error
# fig.add_trace(go.Scatter(x = VT_perror1[:,0], y = VT_perror1[:,1], line = dict(dash = "dash"), marker = dict(color = "green"),  name = "Warped Error"), row = 2, col = 1)
fig.add_trace(go.Scatter(x = VT_perror2[:,0], y = VT_perror2[:,1], line = dict(dash = "dot"), marker = dict(color = "green"), name = "Rewarped Error"), row=2, col = 1)
fig.add_trace(go.Scatter(x = VT_missed_peak_error[:,0], y = VT_missed_peak_error[:,1], line = dict(dash = "dash"), marker = dict(color = "green"), name = "Missed Peak Error"), row = 2, col = 1)
fig.update_layout(title = "Tidal Volume (VT) Percent Error")

In [313]:
# instBR DTW Settings
window = [3,8]
gamma = 2.5 # scaling of shifting deviation penalty
beta = 0.4 # exponential averaging factor

# Repeat for instBR
instBR_live = live_b3_df[["breathTime", "instBR"]].to_numpy()
instBR_pp = pp_b3_df[["breathTime", "instBR"]].to_numpy()

instBR_DTW, instBR_i_to_j, _ = dtw.dtw_distance(instBR_live, instBR_pp, avg_shift= True, window = window, gamma = gamma, beta = beta)
instBR_warped = dtw.remap_query(instBR_live, instBR_pp, instBR_i_to_j)
instBR_i_to_j_fixed = dtw.fix_i_to_j(instBR_i_to_j, instBR_warped)
instBR_rewarped = dtw.remap_query(instBR_live, instBR_pp, instBR_i_to_j_fixed)

instBR_error1 = ptp_abs_error(instBR_warped, instBR_pp)
instBR_error2 = ptp_abs_error(instBR_rewarped, instBR_pp)

instBR_perror1 = ptp_percent_error(instBR_warped, instBR_pp)
instBR_perror2 = ptp_percent_error(instBR_rewarped, instBR_pp)

instBR_error1_frac, instBR_error1_ind = get_frac_in_error(instBR_error1, 5)
instBR_error2_frac, instBR_error2_ind = get_frac_in_error(instBR_error2, 5)

instBR_perror1_frac, instBR_perror1_ind = get_frac_in_error(instBR_perror1, .5)
instBR_perror2_frac, instBR_perror2_ind = get_frac_in_error(instBR_perror2, .5)

instBR_perror2_avg = np.mean(instBR_perror2[:,1])

# print("instBR_error1_frac: ", instBR_error1_frac)
print("instBR_error2_frac: ", instBR_error2_frac)

# print("instBR_perror1_frac: ", instBR_perror1_frac)
print("instBR_perror2_frac: ", instBR_perror2_frac)

print("instBR_perror2_avg: ", instBR_perror2_avg)



instBR_error2_frac:  0.10846560846560846
instBR_perror2_frac:  0.06349206349206349
instBR_perror2_avg:  0.15117399851312097


In [314]:
# plot the results
fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, vertical_spacing = 0.05, subplot_titles=("instBR", "Abs Error"))
# First plot the instBR
fig.add_trace(go.Scatter(x = instBR_live[:,0], y = instBR_live[:,1], marker = dict(color = "blue"),  name = "Live"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = instBR_pp[:,0], y = instBR_pp[:,1], marker = dict(color = "red"), name = "PP"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = instBR_warped[:,0], y = instBR_warped[:,1], line = dict(dash = "dash"),marker = dict(color = "blue"), name = "Warped"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = instBR_rewarped[:,0], y = instBR_rewarped[:,1], line = dict(dash = "dot"),marker = dict(color = "blue"), name = "Rewarped"), row = 1, col = 1)
# Now plot the error
# fig.add_trace(go.Scatter(x = instBR_error1[:,0], y = instBR_error1[:,1], line = dict(dash = "dash"), marker = dict(color = "green"),  name = "Warped Error"), row = 2, col = 1)
fig.add_trace(go.Scatter(x = instBR_error2[:,0], y = instBR_error2[:,1], line = dict(dash = "dot"), marker = dict(color = "green"), name = "Rewarped Error"), row=2, col = 1)
fig.update_layout(title = "Instantaneous Breathing Rate (instBR) Absolute Error")


In [315]:
# plot the results
fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, vertical_spacing = 0.05, subplot_titles=("instBR", "Percent Error"))
# First plot the instBR
fig.add_trace(go.Scatter(x = instBR_live[:,0], y = instBR_live[:,1], marker = dict(color = "blue"),  name = "Live"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = instBR_pp[:,0], y = instBR_pp[:,1], marker = dict(color = "red"), name = "PP"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = instBR_warped[:,0], y = instBR_warped[:,1], line = dict(dash = "dash"),marker = dict(color = "blue"), name = "Warped"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = instBR_rewarped[:,0], y = instBR_rewarped[:,1], line = dict(dash = "dot"),marker = dict(color = "blue"), name = "Rewarped"), row = 1, col = 1)
# Now plot the error
# fig.add_trace(go.Scatter(x = instBR_error1[:,0], y = instBR_error1[:,1], line = dict(dash = "dash"), marker = dict(color = "green"),  name = "Warped Error"), row = 2, col = 1)
fig.add_trace(go.Scatter(x = instBR_perror2[:,0], y = instBR_perror2[:,1], line = dict(dash = "dot"), marker = dict(color = "green"), name = "Rewarped Error"), row=2, col = 1)
fig.update_layout(title = "Instantaneous Breathing Rate (instBR) Percent Error")

In [316]:
# RRAvg DTW Settings
window = [3,8]
gamma = 2.5 # scaling of shifting deviation penalty
beta = 0.4 # exponential averaging factor
perror_thresh = 0.2 # percent error threshold

In [317]:
# Repeat for RRAvg
RRAvg_live = live_b3_df[["breathTime", "RRAvg"]].to_numpy()
RRAvg_pp = pp_b3_df[["breathTime", "RRAvg"]].to_numpy()

RRAvg_DTW, RRAvg_i_to_j, _ = dtw.dtw_distance(RRAvg_live, RRAvg_pp, avg_shift= True, window = window, gamma = gamma, beta = beta)
RRAvg_warped = dtw.remap_query(RRAvg_live, RRAvg_pp, RRAvg_i_to_j)
RRAvg_i_to_j_fixed = dtw.fix_i_to_j(RRAvg_i_to_j, RRAvg_warped)
RRAvg_rewarped = dtw.remap_query(RRAvg_live, RRAvg_pp, RRAvg_i_to_j_fixed)
RRAvg_error1 = ptp_abs_error(RRAvg_warped, RRAvg_pp)
RRAvg_error2 = ptp_abs_error(RRAvg_rewarped, RRAvg_pp)
RRAvg_perror1 = ptp_percent_error(RRAvg_warped, RRAvg_pp)
RRAvg_perror2 = ptp_percent_error(RRAvg_rewarped, RRAvg_pp)

RRAvg_perror2_frac, RRAvg_perror2_ind = get_frac_in_error(RRAvg_perror2, perror_thresh)

RRAvg_perror2_avg = np.mean(RRAvg_perror2[:,1])

print(f"RRAvg_perror2_frac (>{perror_thresh}): ", RRAvg_perror2_frac)
print("RRAvg_perror2_avg: ", RRAvg_perror2_avg)



RRAvg_perror2_frac (>0.2):  0.06084656084656084
RRAvg_perror2_avg:  0.06825030614758068


In [318]:
# plot the results
fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, vertical_spacing = 0.05, subplot_titles=("RRAvg", "Error"))
# First plot the RRAvg
fig.add_trace(go.Scatter(x = RRAvg_live[:,0], y = RRAvg_live[:,1], marker = dict(color = "blue"),  name = "Live"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = RRAvg_pp[:,0], y = RRAvg_pp[:,1], marker = dict(color = "red"), name = "PP"), row = 1, col = 1)
# fig.add_trace(go.Scatter(x = RRAvg_warped[:,0], y = RRAvg_warped[:,1], line = dict(dash = "dash"),marker = dict(color = "blue"), name = "Warped"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = RRAvg_rewarped[:,0], y = RRAvg_rewarped[:,1], line = dict(dash = "dot"),marker = dict(color = "blue"), name = "Rewarped"), row = 1, col = 1)
# Now plot the error
# fig.add_trace(go.Scatter(x = RRAvg_error1[:,0], y = RRAvg_error1[:,1], line = dict(dash = "dash"), marker = dict(color = "green"),  name = "Warped Error"), row = 2, col = 1)
fig.add_trace(go.Scatter(x = RRAvg_error2[:,0], y = RRAvg_error2[:,1], line = dict(dash = "dot"), marker = dict(color = "green"), name = "Rewarped Error"), row=2, col = 1)
fig.update_layout(title = "RRAvg")


In [319]:
# RRAvg Percent error plots
fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, vertical_spacing = 0.05, subplot_titles=("RRAvg", "Error"))
# First plot the RRAvg
fig.add_trace(go.Scatter(x = RRAvg_live[:,0], y = RRAvg_live[:,1], marker = dict(color = "blue"),  name = "Live"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = RRAvg_pp[:,0], y = RRAvg_pp[:,1], marker = dict(color = "red"), name = "PP"), row = 1, col = 1)
# fig.add_trace(go.Scatter(x = RRAvg_warped[:,0], y = RRAvg_warped[:,1], line = dict(dash = "dash"),marker = dict(color = "blue"), name = "Warped"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = RRAvg_rewarped[:,0], y = RRAvg_rewarped[:,1], line = dict(dash = "dot"),marker = dict(color = "blue"), name = "Rewarped"), row = 1, col = 1)
# Now plot the error
# fig.add_trace(go.Scatter(x = RRAvg_error1[:,0], y = RRAvg_error1[:,1], line = dict(dash = "dash"), marker = dict(color = "green"),  name = "Warped Error"), row = 2, col = 1)
fig.add_trace(go.Scatter(x = RRAvg_perror2[:,0], y = RRAvg_perror2[:,1], line = dict(dash = "dot"), marker = dict(color = "green"), name = "Rewarped Error"), row=2, col = 1)
fig.update_layout(title = "RRAvg")

In [320]:
# VE DTW Settings
window = [20,60]
gamma = 2.5 # scaling of shifting deviation penalty
beta = 0.4 # exponential averaging factor

# Do the same with VE
VE_live = live_b3_df[["breathTime", "VE"]].to_numpy()
VE_pp = pp_b3_df[["breathTime", "VE"]].to_numpy()
VE_live = VE_live[5:,:]
VE_DTW, VE_i_to_j, _ = dtw.dtw_distance(VE_live, VE_pp, avg_shift= True, window = window, gamma = gamma, beta = beta)
VE_warped = dtw.remap_query(VE_live, VE_pp, VE_i_to_j)
VE_i_to_j_fixed = dtw.fix_i_to_j(VE_i_to_j, VE_warped)
VE_rewarped = dtw.remap_query(VE_live, VE_pp, VE_i_to_j_fixed)
VE_error1 = ptp_abs_error(VE_warped, VE_pp)
VE_error2 = ptp_abs_error(VE_rewarped, VE_pp)

# VE Percent error
VE_perror2 = ptp_percent_error(VE_rewarped, VE_pp)
# VE Percent error average
VE_perror2_avg = np.mean(VE_perror2[:,1])

print("VE Percent Error Average: ", VE_perror2_avg)
#print(VE_perror2)


VE Percent Error Average:  0.13078394487268397


In [321]:
# plot the results
fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, vertical_spacing = 0.05, subplot_titles=("VE", "VE Error"))
# First plot the VE
fig.add_trace(go.Scatter(x = VE_live[:,0], y = VE_live[:,1], marker = dict(color = "blue"),  name = "Live"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = VE_pp[:,0], y = VE_pp[:,1], marker = dict(color = "red"), name = "PP"), row = 1, col = 1)
# fig.add_trace(go.Scatter(x = VE_warped[:,0], y = VE_warped[:,1], line = dict(dash = "dash"),marker = dict(color = "blue"), name = "Warped"), row = 1, col = 1)
fig.add_trace(go.Scatter(x = VE_rewarped[:,0], y = VE_rewarped[:,1], line = dict(dash = "dot"),marker = dict(color = "blue"), name = "Rewarped"), row = 1, col = 1)
# Now plot the error
# fig.add_trace(go.Scatter(x = VE_error1[:,0], y = VE_error1[:,1], line = dict(dash = "dash"), marker = dict(color = "green"),  name = "Warped Error"), row = 2, col = 1)
fig.add_trace(go.Scatter(x = VE_error2[:,0], y = VE_error2[:,1], line = dict(dash = "dot"), marker = dict(color = "green"), name = "Rewarped Error"), row=2, col = 1)
fig.update_layout(title = "Minute Ventilation (VE)")


In [322]:
# plot the results
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, subplot_titles=("VE", "VE Percent Error"))
# First plot the VE
fig.add_trace(go.Scatter(x=VE_live[:, 0], y=VE_live[:, 1], marker=dict(color="blue"), name="Live"), row=1, col=1)
fig.add_trace(go.Scatter(x=VE_pp[:, 0], y=VE_pp[:, 1], marker=dict(color="red"), name="PP"), row=1, col=1)
# fig.add_trace(go.Scatter(x = VE_warped[:,0], y = VE_warped[:,1], line = dict(dash = "dash"),marker = dict(color = "blue"), name = "Warped"), row = 1, col = 1)
fig.add_trace(go.Scatter(x=VE_rewarped[:, 0], y=VE_rewarped[:, 1], line=dict(dash="dot"), marker=dict(color="blue"),
                         name="Rewarped"), row=1, col=1)
# Now plot the error
# fig.add_trace(go.Scatter(x = VE_error1[:,0], y = VE_error1[:,1], line = dict(dash = "dash"), marker = dict(color = "green"),  name = "Warped Error"), row = 2, col = 1)
fig.add_trace(go.Scatter(x=VE_perror2[:, 0], y=VE_perror2[:, 1], line=dict(dash="dot"), marker=dict(color="green"),
                         name="Rewarped Error"), row=2, col=1)
fig.update_layout(title="Minute Ventilation (VE) Percent Error")